# ChromaDB + LocalFileStore에 Parent Document 임베딩 진행

In [1]:
# @title set API key
import os
import getpass
from pprint import pprint
import warnings

warnings.filterwarnings("ignore")

from IPython import get_ipython

if "google.colab" in str(get_ipython()):
    # Running in Google Colab. Please set the UPSTAGE_API_KEY in the Colab Secrets
    from google.colab import userdata
    os.environ["UPSTAGE_API_KEY"] = userdata.get("UPSTAGE_API_KEY")
else:
    # Running locally. Please set the UPSTAGE_API_KEY in the .env file
    from dotenv import load_dotenv

    load_dotenv()

if "UPSTAGE_API_KEY" not in os.environ:
    os.environ["UPSTAGE_API_KEY"] = getpass.getpass("Enter your Upstage API key: ")

In [ ]:
#from langchain.storage import InMemoryStore
#from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.retrievers import ParentDocumentRetriever
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_upstage import UpstageEmbeddings
from langchain_chroma import Chroma
from langchain.storage._lc_store import create_kv_docstore
from langchain.storage.file_system import LocalFileStore


loader = CSVLoader(file_path='final_training.csv',
                   source_column='출처',
                   content_columns=['회의날짜', '국회_대', '회의명', '회수', '차수', '안건', '법안', '질문자_이름',
       '질문', '질문_키워드', '문맥', '답변자_이름', '답변', '답변_키워드'])

docs = loader.load()


# It should create documents smaller than the parent
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)

# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="split_parents", embedding_function=UpstageEmbeddings(model="embedding-passage"), 
    persist_directory='child_DB(Chroma)')

# The storage layer for the parent documents
fs = LocalFileStore('./parent_fs_chroma')
store = create_kv_docstore(fs)
big_chunk_retriever = ParentDocumentRetriever(
vectorstore=vectorstore,
docstore=store,
child_splitter=child_splitter)

## OpenAI 임베딩 활용

In [2]:
#from langchain.storage import InMemoryStore
#from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.retrievers import ParentDocumentRetriever
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_upstage import UpstageEmbeddings
from langchain.embeddings import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain.storage._lc_store import create_kv_docstore
from langchain.storage.file_system import LocalFileStore


loader = CSVLoader(file_path='final_training.csv',
                   source_column='출처',
                   content_columns=['회의날짜', '국회_대', '회의명', '회수', '차수', '안건', '법안', '질문자_이름',
       '질문', '질문_키워드', '문맥', '답변자_이름', '답변', '답변_키워드'])

docs = loader.load()


# It should create documents smaller than the parent
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)

# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="split_parents", embedding_function=OpenAIEmbeddings(model="text-embedding-3-large"), 
    persist_directory='child_DB(Chroma, OpenAI)')

# The storage layer for the parent documents
fs = LocalFileStore('./parent_fs_chroma_OpenAI')
store = create_kv_docstore(fs)
big_chunk_retriever = ParentDocumentRetriever(
vectorstore=vectorstore,
docstore=store,
child_splitter=child_splitter)

/var/folders/g2/j70lmnjx7r338m_8lsfpyd440000gn/T/ipykernel_79343/497589892.py:27: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  collection_name="split_parents", embedding_function=OpenAIEmbeddings(model="text-embedding-3-large"),


In [4]:
from tqdm.notebook import tqdm
#
def embedding(a,b,MAX_BATCH_SIZE=100):
    #
    for i in tqdm(range(a,b,MAX_BATCH_SIZE)):
        print(f"start : {i}")
        i_end = min(len(docs),i+MAX_BATCH_SIZE)
        print(f"end: {i_end}")
        batch = docs[i:i_end]
        try:
            big_chunk_retriever.add_documents(batch, ids=None)
        except ValueError as e:
            print(e)
            #ct = i_end // 2
            temp = batch
            #print(ct)
            big_chunk_retriever.add_documents(temp[:50], ids=None)
            big_chunk_retriever.add_documents(temp[50:], ids=None)
            continue

        print(f"Number of keys stored in the doctstore :{len(list(store.yield_keys()))}")

In [7]:
embedding(0, 5100)

  0%|          | 0/51 [00:00<?, ?it/s]

start : 0
end: 100
Number of keys stored in the doctstore :100
start : 100
end: 200
Number of keys stored in the doctstore :200
start : 200
end: 300
Number of keys stored in the doctstore :300
start : 300
end: 400
Number of keys stored in the doctstore :400
start : 400
end: 500
Number of keys stored in the doctstore :500
start : 500
end: 600
Number of keys stored in the doctstore :600
start : 600
end: 700
Number of keys stored in the doctstore :700
start : 700
end: 800
Number of keys stored in the doctstore :800
start : 800
end: 900
Number of keys stored in the doctstore :900
start : 900
end: 1000
Number of keys stored in the doctstore :1000
start : 1000
end: 1100
Number of keys stored in the doctstore :1100
start : 1100
end: 1200
Number of keys stored in the doctstore :1200
start : 1200
end: 1300
Number of keys stored in the doctstore :1300
start : 1300
end: 1400
Number of keys stored in the doctstore :1400
start : 1400
end: 1500
Number of keys stored in the doctstore :1500
start : 15

In [8]:
embedding(5100, 35231)

  0%|          | 0/302 [00:00<?, ?it/s]

start : 5100
end: 5200
Number of keys stored in the doctstore :5200
start : 5200
end: 5300
Number of keys stored in the doctstore :5300
start : 5300
end: 5400
Number of keys stored in the doctstore :5400
start : 5400
end: 5500
Number of keys stored in the doctstore :5500
start : 5500
end: 5600
Number of keys stored in the doctstore :5600
start : 5600
end: 5700
Number of keys stored in the doctstore :5700
start : 5700
end: 5800
Number of keys stored in the doctstore :5800
start : 5800
end: 5900
Number of keys stored in the doctstore :5900
start : 5900
end: 6000
Number of keys stored in the doctstore :6000
start : 6000
end: 6100
Number of keys stored in the doctstore :6100
start : 6100
end: 6200
Number of keys stored in the doctstore :6200
start : 6200
end: 6300
Number of keys stored in the doctstore :6300
start : 6300
end: 6400
Number of keys stored in the doctstore :6400
start : 6400
end: 6500
Number of keys stored in the doctstore :6500
start : 6500
end: 6600
Number of keys stored in 

In [13]:
sub_docs = vectorstore.similarity_search("한덕수 총리가 무역적자에 관해 언급한 내용은 무엇인가요?")

print("글 길이: {}\n\n".format(len(sub_docs[0].page_content)))
print(sub_docs[0].page_content)

retrieved_docs = big_chunk_retriever.invoke("한덕수 총리가 무역적자에 관해 언급한 내용은 무엇인가요?")

print("글 길이: {}\n\n".format(len(retrieved_docs[0].page_content)))
print(retrieved_docs[0].page_content)

글 길이: 356


회의날짜: 2022년7월26일(화)
국회_대: 21
회의명: 국회본회의
회수: 제398회
차수: 제6차
안건: 경제에 관한 질문
법안: 경제에 관한 질문
질문자_이름: 김경협  의원
질문: 무역적자가 28년 만에 발생한 이유는 무엇이라고 생각하십니까?
질문_키워드: 국제사회, 무역적자
문맥: '무역적자가 28년 만에 발생한 이유는 중국의 경제 성장이 둔화하고 중국의 산업 정책이 변화함에 따라 우리의 중국 수출 퍼텐셜이 줄어들었기 때문이라고 생각합니다. 여기에 중국의 글로벌 서플라이 체인 변화도 영향을 미치고 있습니다. 이로 인해 우리는 미래 수출을 다원화하고 다변화해야 할 필요가 있다고 생각합니다.'
답변자_이름: 한덕수 국무총리
글 길이: 528


회의날짜: 2022년7월26일(화)
국회_대: 21
회의명: 국회본회의
회수: 제398회
차수: 제6차
안건: 경제에 관한 질문
법안: 경제에 관한 질문
질문자_이름: 김경협  의원
질문: 무역적자가 28년 만에 발생한 이유는 무엇이라고 생각하십니까?
질문_키워드: 국제사회, 무역적자
문맥: '무역적자가 28년 만에 발생한 이유는 중국의 경제 성장이 둔화하고 중국의 산업 정책이 변화함에 따라 우리의 중국 수출 퍼텐셜이 줄어들었기 때문이라고 생각합니다. 여기에 중국의 글로벌 서플라이 체인 변화도 영향을 미치고 있습니다. 이로 인해 우리는 미래 수출을 다원화하고 다변화해야 할 필요가 있다고 생각합니다.'
답변자_이름: 한덕수 국무총리
답변: 중국의 경제 성장이 둔화하고 중국의 산업 정책이 변화함에 따라 우리의 중국 수출 퍼텐셜이 줄어들었기 때문이라고 생각합니다. 여기에 중국의 글로벌 서플라이 체인 변화도 영향을 미치고 있습니다. 이로 인해 우리는 미래 수출을 다원화하고 다변화해야 할 필요가 있다고 생각합니다.
답변_키워드: 중국, 원칙


In [14]:
sub_docs = vectorstore.similarity_search("오픈 프라이머리가 무엇인가요?")

print("글 길이: {}\n\n".format(len(sub_docs[0].page_content)))
print(sub_docs[0].page_content)

retrieved_docs = big_chunk_retriever.invoke("오픈 프라이머리가 무엇인가요?")

print("글 길이: {}\n\n".format(len(retrieved_docs[0].page_content)))
print(retrieved_docs[0].page_content)

글 길이: 245


문맥: 의무공개매수제도는 특정 대주주에 대한 프리미엄만 부여되고, 일반 대주주에게 혜택이 가지 않아 M&A 저해와 문제 발생 우려로 인해 폐지되었습니다.
답변자_이름: 김주현 금융위원장
답변: 의무공개매수제도는 특정 대주주에 대한 프리미엄만 가고, 일반 대주주에게 혜택이 가지 않아 위기 상황에서 M&A를 저해하고 문제가 많다는 이유로 폐지되었습니다.
답변_키워드: 의무공개매수제도,  특정 대주주,  프리미엄,  일반 대주주,   M&A
글 길이: 422


회의날짜: 2022년10월6일(목)
국회_대: 21
회의명: 정무위원회
회수: 2022
차수: 2022년10월06일
안건: 감사개시
법안: 감사개시
질문자_이름: 강병원 위원
질문: 의무공개매수제도 폐지 이유는 무엇인가요?
질문_키워드: 경영권,  경영권 프리미엄,  선진국,  의무공개매수제도,  일반 투자자 보호
문맥: 의무공개매수제도는 특정 대주주에 대한 프리미엄만 부여되고, 일반 대주주에게 혜택이 가지 않아 M&A 저해와 문제 발생 우려로 인해 폐지되었습니다.
답변자_이름: 김주현 금융위원장
답변: 의무공개매수제도는 특정 대주주에 대한 프리미엄만 가고, 일반 대주주에게 혜택이 가지 않아 위기 상황에서 M&A를 저해하고 문제가 많다는 이유로 폐지되었습니다.
답변_키워드: 의무공개매수제도,  특정 대주주,  프리미엄,  일반 대주주,   M&A


In [8]:
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_upstage import ChatUpstage
from langchain_community.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name='gpt-4o', temperature=0)
qa_with_source = RetrievalQAWithSourcesChain.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=big_chunk_retriever, return_source_documents=True)

In [10]:
retrieved_docs

[Document(metadata={'source': 'http://likms.assembly.go.kr/record/mhs-10-040-0040.do?conferNum=045004&fileType=PDF', 'row': 1957}, page_content='회의날짜: 2015년4월1일(수)\n국회_대: 19\n회의명: 정치개혁특별위원회\n회수: 제331회\n차수: 제2차\n안건: 현안보고- 중앙선거관리위원회\n법안: 없음\n질문자_이름: 박영선 위원\n질문: 선관위가 오픈 프라이머리의 종류를 고려하고 있을 때, 어떤 형태의 완전 국민경선을 고려하고 있나요?\n질문_키워드: 오픈, 프라이머리, 가능, 답변, 정당, 후보, 추천, 민주성\n문맥: 선관위는 완전 국민경선을 구상하고 있으며, 이는 일반적인 오픈 프라이머리의 한 형태로 당원뿐만 아니라 일반 국민 모두가 어떤 정당의 프라이머리에 자유롭게 참여할 수 있는 방식입니다. 이는 특정 정당의 당원들이 자신들의 정당에만 참여하는 것이 아닌, 당원과 비당원을 포함한 모든 국민이 자율적으로 참여할 수 있는 오픈 프라이머리입니다. 선관위가 고려하고 있는 시스템은 선거인명부를 관에서 직권으로 작성하는 방식을 포함하며, 이는 당원이 아닌 국민들까지도 모든 정당의 오픈 프라이머리에 참여할 수 있다는 점에서 완전 국민경선의 개념을 담고 있습니다. 이렇게 함으로써 정당 후보자 추천의 민주성을 강화하고자 하는 선관위의 의도가 반영된 것입니다.\n답변자_이름: 김용희 중앙선거관리위원회 사무총장\n답변: 선관위는 완전 국민경선을 구상하고 있으며, 이는 당원과 일반 국민 모두가 어느 정당의 오픈 프라이머리에 자유롭게 참여할 수 있는 형태로, 선거인명부를 관에서 직권으로 작성하는 방식을 고려하고 있습니다.\n답변_키워드: 오픈, 프라이머리, 참여, 형태, 당원, 정당, 일반'),
 Document(metadata={'source': 'http://likms.assembly.go.kr/record/mhs-10-040-0040.do?con

In [11]:
big_chunk_retriever = ParentDocumentRetriever(
vectorstore=vectorstore,
docstore=store,
child_splitter=child_splitter,
search_kwargs={"k": 5})

In [12]:
sub_docs = vectorstore.similarity_search("오픈 프라이머리")

print("글 길이: {}\n\n".format(len(sub_docs[0].page_content)))
print(sub_docs[0].page_content)

retrieved_docs = big_chunk_retriever.invoke("오픈 프라이머리")

print("글 길이: {}\n\n".format(len(retrieved_docs[0].page_content)))
print(retrieved_docs[0].page_content)

글 길이: 181


답변자_이름: 김용희 중앙선거관리위원회 사무총장
답변: 선관위는 완전 국민경선을 구상하고 있으며, 이는 당원과 일반 국민 모두가 어느 정당의 오픈 프라이머리에 자유롭게 참여할 수 있는 형태로, 선거인명부를 관에서 직권으로 작성하는 방식을 고려하고 있습니다.
답변_키워드: 오픈, 프라이머리, 참여, 형태, 당원, 정당, 일반
글 길이: 750


회의날짜: 2015년4월1일(수)
국회_대: 19
회의명: 정치개혁특별위원회
회수: 제331회
차수: 제2차
안건: 현안보고- 중앙선거관리위원회
법안: 없음
질문자_이름: 박영선 위원
질문: 선관위가 오픈 프라이머리의 종류를 고려하고 있을 때, 어떤 형태의 완전 국민경선을 고려하고 있나요?
질문_키워드: 오픈, 프라이머리, 가능, 답변, 정당, 후보, 추천, 민주성
문맥: 선관위는 완전 국민경선을 구상하고 있으며, 이는 일반적인 오픈 프라이머리의 한 형태로 당원뿐만 아니라 일반 국민 모두가 어떤 정당의 프라이머리에 자유롭게 참여할 수 있는 방식입니다. 이는 특정 정당의 당원들이 자신들의 정당에만 참여하는 것이 아닌, 당원과 비당원을 포함한 모든 국민이 자율적으로 참여할 수 있는 오픈 프라이머리입니다. 선관위가 고려하고 있는 시스템은 선거인명부를 관에서 직권으로 작성하는 방식을 포함하며, 이는 당원이 아닌 국민들까지도 모든 정당의 오픈 프라이머리에 참여할 수 있다는 점에서 완전 국민경선의 개념을 담고 있습니다. 이렇게 함으로써 정당 후보자 추천의 민주성을 강화하고자 하는 선관위의 의도가 반영된 것입니다.
답변자_이름: 김용희 중앙선거관리위원회 사무총장
답변: 선관위는 완전 국민경선을 구상하고 있으며, 이는 당원과 일반 국민 모두가 어느 정당의 오픈 프라이머리에 자유롭게 참여할 수 있는 형태로, 선거인명부를 관에서 직권으로 작성하는 방식을 고려하고 있습니다.
답변_키워드: 오픈, 프라이머리, 참여, 형태, 당원, 정당, 일반


In [13]:
retrieved_docs

[Document(metadata={'source': 'http://likms.assembly.go.kr/record/mhs-10-040-0040.do?conferNum=045004&fileType=PDF', 'row': 1957}, page_content='회의날짜: 2015년4월1일(수)\n국회_대: 19\n회의명: 정치개혁특별위원회\n회수: 제331회\n차수: 제2차\n안건: 현안보고- 중앙선거관리위원회\n법안: 없음\n질문자_이름: 박영선 위원\n질문: 선관위가 오픈 프라이머리의 종류를 고려하고 있을 때, 어떤 형태의 완전 국민경선을 고려하고 있나요?\n질문_키워드: 오픈, 프라이머리, 가능, 답변, 정당, 후보, 추천, 민주성\n문맥: 선관위는 완전 국민경선을 구상하고 있으며, 이는 일반적인 오픈 프라이머리의 한 형태로 당원뿐만 아니라 일반 국민 모두가 어떤 정당의 프라이머리에 자유롭게 참여할 수 있는 방식입니다. 이는 특정 정당의 당원들이 자신들의 정당에만 참여하는 것이 아닌, 당원과 비당원을 포함한 모든 국민이 자율적으로 참여할 수 있는 오픈 프라이머리입니다. 선관위가 고려하고 있는 시스템은 선거인명부를 관에서 직권으로 작성하는 방식을 포함하며, 이는 당원이 아닌 국민들까지도 모든 정당의 오픈 프라이머리에 참여할 수 있다는 점에서 완전 국민경선의 개념을 담고 있습니다. 이렇게 함으로써 정당 후보자 추천의 민주성을 강화하고자 하는 선관위의 의도가 반영된 것입니다.\n답변자_이름: 김용희 중앙선거관리위원회 사무총장\n답변: 선관위는 완전 국민경선을 구상하고 있으며, 이는 당원과 일반 국민 모두가 어느 정당의 오픈 프라이머리에 자유롭게 참여할 수 있는 형태로, 선거인명부를 관에서 직권으로 작성하는 방식을 고려하고 있습니다.\n답변_키워드: 오픈, 프라이머리, 참여, 형태, 당원, 정당, 일반'),
 Document(metadata={'source': 'http://likms.assembly.go.kr/record/mhs-10-040-0040.do?con

In [99]:
df.columns

Index(['출처', 'id', '회의날짜', '국회_대', '회의명', '회수', '차수', '안건', '법안', '질문자_이름',
       '질문', '질문_키워드', '문맥', '답변자_이름', '답변', '답변_키워드'],
      dtype='object')

In [114]:
import pandas as pd
df = pd.read_csv('final_training.csv', index_col=0)

df = df[['출처', 'id', '회의날짜', '질문자_이름',
       '질문', '질문_키워드', '답변자_이름', '문맥(context)', '실제 답변', '답변_키워드', '국회_대', '회의명', '회수', '차수', '안건', '법안']]
df.head()

,출처,id,회의날짜,질문자_이름,질문,질문_키워드,답변자_이름,문맥(context),실제 답변,답변_키워드,국회_대,회의명,회수,차수,안건,법안
0,http://likms.assembly.go.kr/record/mhs-10-040-...,46416,2016년9월23일(금),전재수 의원,"미르재단 설립 허가 신청서는 어디서 접수되었으며, 그 처리과정에 대해 어떠한 절차가...","미르재단, 설립, 허가, 대중문화산업과, 직원, 세종시, 청사, 근무",조윤선 문화체육관광부 장관,미르재단 설립 허가 신청서의 접수와 처리과정은 문화체육관광부의 대중문화산업과에서 담...,"미르재단 설립 허가 신청은 대중문화산업과에서 접수되었고, 이메일을 통한 초기 신청 ...","서울, 이메일, 접수, 서류, 직원, 출장",20,국회본회의,제346회,제8차,교육․사회․문화에 관한 질문,없음
1,http://likms.assembly.go.kr/record/mhs-10-040-...,47966,2018년2월07일(수),유은혜 의원,추가 조사에서 교수의 직계가족을 대상으로 한정한 것에 따른 부정행위의 미비한 조사 ...,"추가, 조사, 대상, 직계가족, 친인척, 자녀, 공저자, 제외",김상곤 교육부 부총리겸교육부장관,"추가 조사에서 직계자녀를 우선 대상으로 삼았지만, 부총리를 단장으로 하는 대입공정성...","네, 추가 조사에서 직계자녀를 우선 대상으로 삼았지만, 부총리를 단장으로 하는 대입...","직계자녀, 대입공정성강화추진점검단, 점검단, 조사",20,국회본회의,제356회,제7차,교육․사회․문화에 관한 질문(계속),없음
2,http://likms.assembly.go.kr/record/mhs-10-040-...,49896,2020년3월04일(수),홍문표 의원,홍성군에서 발생하는 폐기물 처리장 건설 및 지역 주민 반대에 대한 대응은 어떻게 할...,"폐기물, 처리장",조명래 환경부 장관,지역 주민들의 반대에도 불구하고 홍성군 갈산면 오두리에 폐기물 처리장 건설 계획이 ...,"현재 폐기물 정책 전반의 대전환을 고려하고, 발생지 처리 원칙을 포함한 폐기물 관리...","폐기물, 처리, 지역주민",20,국회본회의,제376회,제7차,교육․사회․문화에 관한 질문,없음
3,http://likms.assembly.go.kr/record/mhs-10-040-...,49412,2019년9월27일(금),민홍철 의원,왜 함박도와 관련된 지역에 대해 40년 동안 현장조사나 초소 설치와 같은 행위가 이...,"함박도, 현장조사",정경두 국방부 장관,함박도와 관련한 40년간의 현장조사 또는 초소 설치 등의 행위가 이루어지지 않은 이...,현장조사 및 행정조치 부재는 관할권 문제와 관련이 있습니다. 정전협정 시점부터 북한...,"관할권, 문제, 확인, 행정조치, 국방부, 잘못, 현장조사, 검증팀, ...",20,국회본회의,제371회,제3차,외교․통일․안보에 관한 질문,없음
4,http://likms.assembly.go.kr/record/mhs-10-040-...,49012,2019년3월20일(수),윤상현 의원,핵무기 확산 가능성에 대비하여 어떠한 대응책과 국제사회와의 협조 방안을 고려하고 있나요?,"핵무기, 확산, 가능성, 지구적, 재앙, 담보",정경두 국방부 장관,현대의 글로벌 안보 상황에서 핵무기의 확산은 심각한 위협으로 다가올 수 있습니다. ...,"핵 확산을 단호하게 허용할 수 없다는 입장을 갖고 있으며, 해당 위협이 나타날 경...","핵, 확산, 단호, 허용, 국제사회, 공조",20,국회본회의,제367회,제6차,외교․통일․안보에 관한 질문,없음


In [117]:
df.to_csv('final_training.csv')

In [118]:
from langchain_community.document_loaders.csv_loader import CSVLoader
loader = CSVLoader(file_path='final_training.csv',
                   source_column='출처',
                   content_columns=['회의날짜', '질문자_이름',
       '질문', '질문_키워드', '답변자_이름', '문맥(context)', '실제 답변', '답변_키워드', '국회_대', '회의명', '회수', '차수', '안건', '법안'])

docs = loader.load()

In [119]:
print(docs[0])

page_content='회의날짜: 2016년9월23일(금)
질문자_이름: 전재수 의원
질문: 미르재단 설립 허가 신청서는 어디서 접수되었으며, 그 처리과정에 대해 어떠한 절차가 따라졌는지 설명해 주실 수 있나요?
질문_키워드: 미르재단,  설립,  허가,  대중문화산업과,  직원,  세종시,  청사,  근무
답변자_이름: 조윤선 문화체육관광부 장관
문맥(context): 미르재단 설립 허가 신청서의 접수와 처리과정은 문화체육관광부의 대중문화산업과에서 담당되었으며, 이 과정은 전형적인 부처의 업무 프로세스를 따랐습니다. 초기에 신청서는 이메일로 접수되었고, 이후 서울에 출장 중이던 담당 직원이 실물 서류를 받아서 처리하게 되었습니다. 이 직원은 문체부 서울 사무실에서 법인 설립 허가서를 받아 그날 저녁 나루, 즉 문서 등록 시스템에 기안을 올렸습니다. 직후 해당 사무관과 과장이 세종시 문체부 본부에서 각각 결재를 진행했습니다. 이러한 신속한 결재 과정은 비일비재하게 야근을 하는 공무원들의 업무 패턴 중 하나로 볼 수 있으나, 이 경우 그 속도와 시간대가 일반적인 관행과 다소 차이가 있어 눈길을 끌었습니다.
실제 답변: 미르재단 설립 허가 신청은 대중문화산업과에서 접수되었고, 이메일을 통한 초기 신청 후 서울 출장 중인 직원이 직접 실질적인 서류를 서울에서 전달받은 것으로 알고 있습니다. 처리과정은 해당 부서의 업무 절차에 따라 진행되었습니다.
답변_키워드: 서울,  이메일,  접수,  서류,  직원,  출장
국회_대: 20
회의명: 국회본회의
회수: 제346회
차수: 제8차
안건: 교육․사회․문화에 관한 질문
법안: 없음' metadata={'source': 'http://likms.assembly.go.kr/record/mhs-10-040-0040.do?conferNum=046416&fileType=PDF', 'row': 0}


## 문서 splitter를 custom으로 설정하여 문서를 나누자
- 회의 날짜 등 기본 설명(안건, 법안)
- 질의 관련
- 문맥 및 답변 관련

In [120]:
from langchain.text_splitter import TextSplitter
from langchain.schema import Document

class CustomColumnSplitter(TextSplitter):
    def string_to_dict(self, input_string):
        """
        Convert a given string with key-value pairs separated by newlines into a dictionary.
        """
        result_dict = {}
        for line in input_string.split("\n"):
            if ": " in line:  # Ensure valid key-value format
                key, value = line.split(": ", 1)  # Split only on the first occurrence of ": "
                result_dict[key.strip()] = value.strip()
        return result_dict

    def split_text(self, text):
        """
        text를 열 데이터로 분리하여 문서를 생성합니다.
        """
        # 텍스트를 딕셔너리로 변환 (CSV 행 형태로 가정)
        row_data = self.string_to_dict(text)  # 문자열을 딕셔너리로 변환 (Document에서 가져온 데이터)

        # 열 기준으로 나누기
        doc1 = {
            "회의날짜": row_data.get("회의날짜"),
            "국회_대": row_data.get("국회_대"),
            "회의명": row_data.get("회의명"),
            "회수": row_data.get("회수"),
            "차수": row_data.get("차수"),
            "안건": row_data.get("안건"),
            "법안": row_data.get("법안"),
        }
        doc2 = {
            "질문자_이름": row_data.get("질문자_이름"),
            "질문": row_data.get("질문"),
            "질문_키워드": row_data.get("질문_키워드"),
        }
        doc3 = {
            "답변자_이름": row_data.get("답변자_이름"),
            "문맥(context)": row_data.get("문맥(context)"),
            "실제 답변": row_data.get("실제 답변"),
            "답변_키워드": row_data.get("답변_키워드"),
        }

        # 반환할 문서 리스트
        return [doc1, doc2, doc3]

    def split_documents(self, documents):
        split_docs = []
        for document in documents:
            # 각 Document의 내용을 split_text로 처리
            sub_docs = self.split_text(document.page_content)
            for sub_doc in sub_docs:
                # 새롭게 분리된 데이터를 Document 객체로 변환
                split_docs.append(
                    Document(
                        page_content=str(sub_doc),  # 문자열로 변환
                        metadata=document.metadata,  # 기존 문서의 메타데이터 유지
                    )
                )
        return split_docs


# Splitter 적용
splitter = CustomColumnSplitter()
split_documents = []

for doc in docs:
    split_documents.extend([
        Document(page_content=str(part)) for part in splitter.split_text(doc.page_content)
    ])

# 결과 출력
for idx, doc in enumerate(split_documents, 1):
    print(f"Document {idx}:")
    print(doc.page_content)
    print(doc.metadata)
    if idx == 5:
        break


Document 1:
{'회의날짜': '2016년9월23일(금)', '국회_대': '20', '회의명': '국회본회의', '회수': '제346회', '차수': '제8차', '안건': '교육․사회․문화에 관한 질문', '법안': '없음'}
{}
Document 2:
{'질문자_이름': '전재수 의원', '질문': '미르재단 설립 허가 신청서는 어디서 접수되었으며, 그 처리과정에 대해 어떠한 절차가 따라졌는지 설명해 주실 수 있나요?', '질문_키워드': '미르재단,  설립,  허가,  대중문화산업과,  직원,  세종시,  청사,  근무'}
{}
Document 3:
{'답변자_이름': '조윤선 문화체육관광부 장관', '문맥(context)': '미르재단 설립 허가 신청서의 접수와 처리과정은 문화체육관광부의 대중문화산업과에서 담당되었으며, 이 과정은 전형적인 부처의 업무 프로세스를 따랐습니다. 초기에 신청서는 이메일로 접수되었고, 이후 서울에 출장 중이던 담당 직원이 실물 서류를 받아서 처리하게 되었습니다. 이 직원은 문체부 서울 사무실에서 법인 설립 허가서를 받아 그날 저녁 나루, 즉 문서 등록 시스템에 기안을 올렸습니다. 직후 해당 사무관과 과장이 세종시 문체부 본부에서 각각 결재를 진행했습니다. 이러한 신속한 결재 과정은 비일비재하게 야근을 하는 공무원들의 업무 패턴 중 하나로 볼 수 있으나, 이 경우 그 속도와 시간대가 일반적인 관행과 다소 차이가 있어 눈길을 끌었습니다.', '실제 답변': '미르재단 설립 허가 신청은 대중문화산업과에서 접수되었고, 이메일을 통한 초기 신청 후 서울 출장 중인 직원이 직접 실질적인 서류를 서울에서 전달받은 것으로 알고 있습니다. 처리과정은 해당 부서의 업무 절차에 따라 진행되었습니다.', '답변_키워드': '서울,  이메일,  접수,  서류,  직원,  출장'}
{}
Document 4:
{'회의날짜': '2018년2월07일(수)', '국회_대': '20', '회의명': '국회본회의', '회수': '제356회', '차수':

In [65]:
splitter.split_documents(docs)

[Document(metadata={'source': 'http://likms.assembly.go.kr/record/mhs-10-040-0040.do?conferNum=046416&fileType=PDF', 'row': 0}, page_content="{'회의날짜': '2016년9월23일(금)', '국회_대': '20', '회의명': '국회본회의', '회수': '제346회', '차수': '제8차', '안건': '교육․사회․문화에 관한 질문', '법안': '없음'}"),
 Document(metadata={'source': 'http://likms.assembly.go.kr/record/mhs-10-040-0040.do?conferNum=046416&fileType=PDF', 'row': 0}, page_content="{'질문자_이름': '전재수 의원', '질문': '미르재단 설립 허가 신청서는 어디서 접수되었으며, 그 처리과정에 대해 어떠한 절차가 따라졌는지 설명해 주실 수 있나요?', '질문_키워드': '미르재단,  설립,  허가,  대중문화산업과,  직원,  세종시,  청사,  근무'}"),
 Document(metadata={'source': 'http://likms.assembly.go.kr/record/mhs-10-040-0040.do?conferNum=046416&fileType=PDF', 'row': 0}, page_content="{'답변자_이름': '조윤선 문화체육관광부 장관', '문맥': '미르재단 설립 허가 신청서의 접수와 처리과정은 문화체육관광부의 대중문화산업과에서 담당되었으며, 이 과정은 전형적인 부처의 업무 프로세스를 따랐습니다. 초기에 신청서는 이메일로 접수되었고, 이후 서울에 출장 중이던 담당 직원이 실물 서류를 받아서 처리하게 되었습니다. 이 직원은 문체부 서울 사무실에서 법인 설립 허가서를 받아 그날 저녁 나루, 즉 문서 등록 시스템에 기안을 올렸습니다. 직후 해당 사무관과 과장이 세종시 문체부 본부에서 각각 결재를 진행했습

In [107]:
from langchain.retrievers import ParentDocumentRetriever
from langchain_upstage import UpstageEmbeddings
from langchain_chroma import Chroma
from langchain.storage._lc_store import create_kv_docstore
from langchain.storage.file_system import LocalFileStore
# It should create documents smaller than the parent
child_splitter = CustomColumnSplitter()

# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="split_parents", embedding_function=UpstageEmbeddings(model="embedding-passage"),
    persist_directory='child_DB(Chroma, Upstage, Custom4)')

# The storage layer for the parent documents
fs = LocalFileStore('./parent_fs_chroma_Upstage_Custom4')
store = create_kv_docstore(fs)
big_chunk_retriever = ParentDocumentRetriever(
vectorstore=vectorstore,
docstore=store,
child_splitter=child_splitter)

In [108]:
from tqdm.notebook import tqdm
#
def embedding(a,b,MAX_BATCH_SIZE=100):
    #
    for i in tqdm(range(a,b,MAX_BATCH_SIZE)):
        print(f"start : {i}")
        i_end = min(len(docs),i+MAX_BATCH_SIZE)
        print(f"end: {i_end}")
        batch = docs[i:i_end]
        try:
            big_chunk_retriever.add_documents(batch, ids=None)
        except ValueError as e:
            print(e)
            #ct = i_end // 2
            temp = batch
            #print(ct)
            big_chunk_retriever.add_documents(temp[:50], ids=None)
            big_chunk_retriever.add_documents(temp[50:], ids=None)
            continue

        print(f"Number of keys stored in the doctstore :{len(list(store.yield_keys()))}")

### embedding

In [109]:
embedding(0, 5100)

  0%|          | 0/51 [00:00<?, ?it/s]

start : 0
end: 100
Number of keys stored in the doctstore :100
start : 100
end: 200
Number of keys stored in the doctstore :200
start : 200
end: 300
Number of keys stored in the doctstore :300
start : 300
end: 400
Number of keys stored in the doctstore :400
start : 400
end: 500
Number of keys stored in the doctstore :500
start : 500
end: 600
Number of keys stored in the doctstore :600
start : 600
end: 700
Number of keys stored in the doctstore :700
start : 700
end: 800
Number of keys stored in the doctstore :800
start : 800
end: 900
Number of keys stored in the doctstore :900
start : 900
end: 1000
Number of keys stored in the doctstore :1000
start : 1000
end: 1100
Number of keys stored in the doctstore :1100
start : 1100
end: 1200
Number of keys stored in the doctstore :1200
start : 1200
end: 1300
Number of keys stored in the doctstore :1300
start : 1300
end: 1400
Number of keys stored in the doctstore :1400
start : 1400
end: 1500
Number of keys stored in the doctstore :1500
start : 15

In [110]:
embedding(5100, 2*5100)

  0%|          | 0/51 [00:00<?, ?it/s]

start : 5100
end: 5200
Number of keys stored in the doctstore :5200
start : 5200
end: 5300
Number of keys stored in the doctstore :5300
start : 5300
end: 5400
Number of keys stored in the doctstore :5400
start : 5400
end: 5500
Number of keys stored in the doctstore :5500
start : 5500
end: 5600
Number of keys stored in the doctstore :5600
start : 5600
end: 5700
Number of keys stored in the doctstore :5700
start : 5700
end: 5800
Number of keys stored in the doctstore :5800
start : 5800
end: 5900
Number of keys stored in the doctstore :5900
start : 5900
end: 6000
Number of keys stored in the doctstore :6000
start : 6000
end: 6100
Number of keys stored in the doctstore :6100
start : 6100
end: 6200
Number of keys stored in the doctstore :6200
start : 6200
end: 6300
Number of keys stored in the doctstore :6300
start : 6300
end: 6400
Number of keys stored in the doctstore :6400
start : 6400
end: 6500
Number of keys stored in the doctstore :6500
start : 6500
end: 6600
Number of keys stored in 

In [111]:
embedding(2*5100, 3*5100)

  0%|          | 0/51 [00:00<?, ?it/s]

start : 10200
end: 10300
Number of keys stored in the doctstore :10300
start : 10300
end: 10400
Number of keys stored in the doctstore :10400
start : 10400
end: 10500
Number of keys stored in the doctstore :10500
start : 10500
end: 10600
Number of keys stored in the doctstore :10600
start : 10600
end: 10700
Number of keys stored in the doctstore :10700
start : 10700
end: 10800
Number of keys stored in the doctstore :10800
start : 10800
end: 10900
Number of keys stored in the doctstore :10900
start : 10900
end: 11000
Number of keys stored in the doctstore :11000
start : 11000
end: 11100
Number of keys stored in the doctstore :11100
start : 11100
end: 11200
Number of keys stored in the doctstore :11200
start : 11200
end: 11300
Number of keys stored in the doctstore :11300
start : 11300
end: 11400
Number of keys stored in the doctstore :11400
start : 11400
end: 11500
Number of keys stored in the doctstore :11500
start : 11500
end: 11600
Number of keys stored in the doctstore :11600
start 

**여기서부터 다시!!!**

In [112]:
embedding(3*5100, 4*5100)

  0%|          | 0/51 [00:00<?, ?it/s]

start : 15300
end: 15400
Number of keys stored in the doctstore :15400
start : 15400
end: 15500
Number of keys stored in the doctstore :15500
start : 15500
end: 15600


KeyboardInterrupt: 

In [124]:
embedding(15500, 4*5100)

  0%|          | 0/49 [00:00<?, ?it/s]

start : 15500
end: 15600
Number of keys stored in the doctstore :15600
start : 15600
end: 15700
Number of keys stored in the doctstore :15700
start : 15700
end: 15800
Number of keys stored in the doctstore :15800
start : 15800
end: 15900
Number of keys stored in the doctstore :15900
start : 15900
end: 16000
Number of keys stored in the doctstore :16000
start : 16000
end: 16100
Number of keys stored in the doctstore :16100
start : 16100
end: 16200
Number of keys stored in the doctstore :16200
start : 16200
end: 16300
Number of keys stored in the doctstore :16300
start : 16300
end: 16400
Number of keys stored in the doctstore :16400
start : 16400
end: 16500
Number of keys stored in the doctstore :16500
start : 16500
end: 16600
Number of keys stored in the doctstore :16600
start : 16600
end: 16700
Number of keys stored in the doctstore :16700
start : 16700
end: 16800
Number of keys stored in the doctstore :16800
start : 16800
end: 16900


KeyboardInterrupt: 

In [78]:
embedding(4*5100, 5*5100)

  0%|          | 0/51 [00:00<?, ?it/s]

start : 20400
end: 20500
Number of keys stored in the doctstore :20500
start : 20500
end: 20600
Number of keys stored in the doctstore :20600
start : 20600
end: 20700
Number of keys stored in the doctstore :20700
start : 20700
end: 20800
Number of keys stored in the doctstore :20800
start : 20800
end: 20900
Number of keys stored in the doctstore :20900
start : 20900
end: 21000
Number of keys stored in the doctstore :21000
start : 21000
end: 21100
Number of keys stored in the doctstore :21100
start : 21100
end: 21200
Number of keys stored in the doctstore :21200
start : 21200
end: 21300
Number of keys stored in the doctstore :21300
start : 21300
end: 21400
Number of keys stored in the doctstore :21400
start : 21400
end: 21500
Number of keys stored in the doctstore :21500
start : 21500
end: 21600
Number of keys stored in the doctstore :21600
start : 21600
end: 21700
Number of keys stored in the doctstore :21700
start : 21700
end: 21800
Number of keys stored in the doctstore :21800
start 

In [79]:
embedding(5*5100, 6*5100)

  0%|          | 0/51 [00:00<?, ?it/s]

start : 25500
end: 25600
Number of keys stored in the doctstore :25600
start : 25600
end: 25700
Number of keys stored in the doctstore :25700
start : 25700
end: 25800
Number of keys stored in the doctstore :25800
start : 25800
end: 25900
Number of keys stored in the doctstore :25900
start : 25900
end: 26000
Number of keys stored in the doctstore :26000
start : 26000
end: 26100
Number of keys stored in the doctstore :26100
start : 26100
end: 26200
Number of keys stored in the doctstore :26200
start : 26200
end: 26300
Number of keys stored in the doctstore :26300
start : 26300
end: 26400
Number of keys stored in the doctstore :26400
start : 26400
end: 26500
Number of keys stored in the doctstore :26500
start : 26500
end: 26600
Number of keys stored in the doctstore :26600
start : 26600
end: 26700
Number of keys stored in the doctstore :26700
start : 26700
end: 26800
Number of keys stored in the doctstore :26800
start : 26800
end: 26900
Number of keys stored in the doctstore :26900
start 

In [80]:
embedding(6*5100, 35231)

  0%|          | 0/47 [00:00<?, ?it/s]

start : 30600
end: 30700
Number of keys stored in the doctstore :30700
start : 30700
end: 30800
Number of keys stored in the doctstore :30800
start : 30800
end: 30900
Number of keys stored in the doctstore :30900
start : 30900
end: 31000
Number of keys stored in the doctstore :31000
start : 31000
end: 31100
Number of keys stored in the doctstore :31100
start : 31100
end: 31200
Number of keys stored in the doctstore :31200
start : 31200
end: 31300
Number of keys stored in the doctstore :31300
start : 31300
end: 31400
Number of keys stored in the doctstore :31400
start : 31400
end: 31500
Number of keys stored in the doctstore :31500
start : 31500
end: 31600
Number of keys stored in the doctstore :31600
start : 31600
end: 31700
Number of keys stored in the doctstore :31700
start : 31700
end: 31800
Number of keys stored in the doctstore :31800
start : 31800
end: 31900
Number of keys stored in the doctstore :31900
start : 31900
end: 32000
Number of keys stored in the doctstore :32000
start 

KeyboardInterrupt: 

In [82]:
len(list(store.yield_keys()))

32600

In [83]:
embedding(32600, 35231)

  0%|          | 0/27 [00:00<?, ?it/s]

start : 32600
end: 32700
Number of keys stored in the doctstore :32700
start : 32700
end: 32800
Number of keys stored in the doctstore :32800
start : 32800
end: 32900
Number of keys stored in the doctstore :32900
start : 32900
end: 33000
Number of keys stored in the doctstore :33000
start : 33000
end: 33100
Number of keys stored in the doctstore :33100
start : 33100
end: 33200
Number of keys stored in the doctstore :33200
start : 33200
end: 33300
Number of keys stored in the doctstore :33300
start : 33300
end: 33400
Number of keys stored in the doctstore :33400
start : 33400
end: 33500
Number of keys stored in the doctstore :33500
start : 33500
end: 33600
Number of keys stored in the doctstore :33600
start : 33600
end: 33700
Number of keys stored in the doctstore :33700
start : 33700
end: 33800
Number of keys stored in the doctstore :33800
start : 33800
end: 33900
Number of keys stored in the doctstore :33900
start : 33900
end: 34000
Number of keys stored in the doctstore :34000
start 

### RAG 진행

In [85]:
def RAG_experiment(query):
    sub_docs = vectorstore.similarity_search(query)

    print("Sub Document")
    print("글 길이: {}\n\n".format(len(sub_docs[0].page_content)))
    print(sub_docs[0].page_content)

    retrieved_docs = big_chunk_retriever.invoke(query)
    print("Retrieved Document")
    print("글 길이: {}\n\n".format(len(retrieved_docs[0].page_content)))
    print(retrieved_docs[0].page_content)

In [86]:
RAG_experiment("한덕수 총리가 무역적자에 관해 언급한 내용은 무엇인가요?")

Sub Document
글 길이: 389


{'답변자_이름': '한덕수 국무총리', '문맥': "'무역적자가 28년 만에 발생한 이유는 중국의 경제 성장이 둔화하고 중국의 산업 정책이 변화함에 따라 우리의 중국 수출 퍼텐셜이 줄어들었기 때문이라고 생각합니다. 여기에 중국의 글로벌 서플라이 체인 변화도 영향을 미치고 있습니다. 이로 인해 우리는 미래 수출을 다원화하고 다변화해야 할 필요가 있다고 생각합니다.'", '답변': '중국의 경제 성장이 둔화하고 중국의 산업 정책이 변화함에 따라 우리의 중국 수출 퍼텐셜이 줄어들었기 때문이라고 생각합니다. 여기에 중국의 글로벌 서플라이 체인 변화도 영향을 미치고 있습니다. 이로 인해 우리는 미래 수출을 다원화하고 다변화해야 할 필요가 있다고 생각합니다.', '답변_키워드': '중국, 원칙'}
Retrieved Document
글 길이: 528


회의날짜: 2022년7월26일(화)
국회_대: 21
회의명: 국회본회의
회수: 제398회
차수: 제6차
안건: 경제에 관한 질문
법안: 경제에 관한 질문
질문자_이름: 김경협  의원
질문: 무역적자가 28년 만에 발생한 이유는 무엇이라고 생각하십니까?
질문_키워드: 국제사회, 무역적자
문맥: '무역적자가 28년 만에 발생한 이유는 중국의 경제 성장이 둔화하고 중국의 산업 정책이 변화함에 따라 우리의 중국 수출 퍼텐셜이 줄어들었기 때문이라고 생각합니다. 여기에 중국의 글로벌 서플라이 체인 변화도 영향을 미치고 있습니다. 이로 인해 우리는 미래 수출을 다원화하고 다변화해야 할 필요가 있다고 생각합니다.'
답변자_이름: 한덕수 국무총리
답변: 중국의 경제 성장이 둔화하고 중국의 산업 정책이 변화함에 따라 우리의 중국 수출 퍼텐셜이 줄어들었기 때문이라고 생각합니다. 여기에 중국의 글로벌 서플라이 체인 변화도 영향을 미치고 있습니다. 이로 인해 우리는 미래 수출을 다원화하고 다변화해야 할 필요가 있다고 생각합니다.
답변_키워드: 중국, 원칙


In [91]:
RAG_experiment("공공의료대학에 대한 정부의 입장은 무엇인가요?")

Sub Document
글 길이: 906


{'답변자_이름': '정세균 국무총리', '문맥': '정부와 국회 양쪽에서 공공의료대학 설립 법안의 통과에 대한 노력의 중요성은 공감대가 형성되어 있으나 실제로 법안이 통과되지 못하는 현 상황에 대해 유감의 목소리가 커지고 있습니다. 이를 통해 필수 의료분야, 특히 감염병과 같은 공중보건 위기에 대비하는 의료 전문가를 양성하는 기관의 중요성 또한 강조되고 있습니다. 사회적인 필요성과 과거의 경험 (메르스 사태 등)으로 볼 때, 이런 전문의를 양성하는 공공의료대학이 필요한 시점임에도 불구하고, 정치적 의사 결정 과정의 지연으로 인해 여러 문제가 발생하고 있습니다. 국회는 법안을 입법화해야 할 책임이 있는 기관으로, 과거에는 다수결의 원칙이 어느 정도 작동하여 중요 법안들이 통과될 수 있었지만, 현재는 몇몇 의원들의 반대로 입법 과정이 지연되거나 불발되는 상황이 발생하고 있습니다. 이는 의료 시스템 내의 구조적 문제를 해결하고 질병 대응 역량을 강화하려는 노력에 걸림돌이 되고 있으며, 이에 대해 큰 우려를 나타내고 있는 것입니다. 따라서, 공공의료대학 설립을 통한 의료 시스템의 강화는 국회에서 긴급히 검토하고 통과시켜야 할 사안으로 보이며, 이대로 정책이 지체될 경우 공중보건 위기 상황에서의 대응력이 약화될 수 있음을 의미합니다. 정부는 법안을 적극적으로 옹호하고 필요한 조치를 취해야 할 책임이 있으며, 국회 또한 공공의 이익을 위해 신속하고 효과적으로 기능해야 한다는 점이 강조되고 있습니다.', '답변': '공공의료대학 설립 법안이 국회에서 통과되지 못하는 상황을 유감스럽게 생각하며, 과거의 다수결 원리가 작동하지 않아 입법이 불발하는 문제를 지적하고 있습니다. 공공의료대학 설립은 근본적인 대책으로, 5월 국회에서 반드시 통과시켜야 한다고 주장하고 있습니다.', '답변_키워드': '공공의료대학'}
Retrieved Document
글 길이: 1111


회의날짜: 2020년3월11일(수)
국회_대: 20
회의명: 예산

In [92]:
RAG_experiment("대학 미충원 인원에 관한 발언들로 무엇이 있나요?")

Sub Document
글 길이: 140


{'질문자_이름': '서동용 의원', '질문': '올해 대학 및 전문대의 신입생 미충원 문제가 심각한데, 이에 대한 정부의 대응책이 있을까요?', '질문_키워드': '지방사립대,  지방 거점 국립대,  전문대학,  추가모집,  미충원 인원,  올해'}
Retrieved Document
글 길이: 717


회의날짜: 2021년4월21일(수)
국회_대: 21
회의명: 국회본회의
회수: 제386회
차수: 제3차
안건: 교육․사회․문화에 관한 질문
법안: 교육․사회․문화에 관한 질문
질문자_이름: 서동용 의원
질문: 올해 대학 및 전문대의 신입생 미충원 문제가 심각한데, 이에 대한 정부의 대응책이 있을까요?
질문_키워드: 지방사립대,  지방 거점 국립대,  전문대학,  추가모집,  미충원 인원,  올해
문맥: 올해 대학 및 전문대의 신입생 미충원 문제에 대한 정부의 대응책은 매우 중요한 문제로 여겨지고 있습니다. 이에 정부는 지방대에 대한 중장기 기본계획을 수립하여 지역과 대학이 협력하여 학생들이 지역의 대학에 입학하고 취업할 수 있는 여건을 만들기 위한 사업을 진행 중입니다. 또한, 국회 상임위에서도 여야 위원들이 협의를 통해 대안을 마련하고 있으며, 5월 초경에는 이에 대한 대안을 발표할 예정입니다.
답변자_이름: 유은혜 교육부 부총리겸장관
답변: 이 문제를 심각하게 받아들이고 지난 3월에 여러 관계부처와 협력하여 지방대에 대한 중장기 기본계획을 수립했습니다. 또한, 국회 상임위에서 여야 위원들이 협의를 통해 대안을 마련하고 있으며, 지역과 대학이 협력하여 학생들이 지역의 대학에 입학하고 취업할 수 있는 여건을 만들기 위한 사업을 진행 중입니다. 5월 초경에는 대안을 발표할 예정입니다.
답변_키워드: 비수도권,  지방대,  중장기 기본계획,  국회 상임위,  고등교육
